In [2]:
import numpy as np

In [44]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

axes = np.array([[0, 0, 1], [0, -1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, -1, 0], [0, 0, 1]])
origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

array([[ 0.   ,  0.   ,  0.   ],
       [ 0.   , -0.349,  0.   ],
       [ 0.349,  0.   , -1.571],
       [ 1.569, -1.222, -3.14 ],
       [-1.57 , -0.348,  0.   ],
       [-1.694, -0.362,  0.372],
       [-1.694, -0.326,  0.372]])

In [73]:
def rotate(angles_rpy, vector):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    b = vector / np.linalg.norm(vector)
    
    sin = np.sin
    cos = np.cos
    
    a = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma) - sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma)],
                 [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma)],
                 [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    c = a.dot(b)
    
    return c / np.linalg.norm(c)


def build_jacobian(origins, angles_rpy, axes, endeffector=origins[-1], offset=0):
    j = np.array([np.concatenate([np.cross(rotate(angle, axis), endeffector - origin), rotate(angle, axis)])
                  for origin, angle, axis in zip(origins[offset:], angles_rpy[offset:], axes[offset:])])
    
    return j.T

In [74]:
jacobian = build_jacobian(origins, angles_rpy, axes, offset=1)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

[[-3.670e-01 -4.351e-01 -1.582e-01 -1.320e-02 -6.758e-02  0.000e+00]
 [-0.000e+00 -2.746e-02 -8.609e-05 -2.225e-05 -2.804e-02  0.000e+00]
 [-1.620e-01 -1.583e-01  4.351e-01  2.493e-02 -2.749e-02 -0.000e+00]
 [ 0.000e+00 -3.420e-01  9.398e-01 -2.716e-04 -3.721e-01 -3.241e-01]
 [-1.000e+00  6.965e-05 -2.996e-04  1.000e+00 -1.326e-02  9.388e-01]
 [ 0.000e+00  9.397e-01  3.418e-01  7.486e-04  9.281e-01 -1.164e-01]]


In [75]:
jacobian_inv = np.linalg.inv(jacobian)
print(jacobian_inv)

[[-2.382e+00 -1.618e+03 -6.906e-01  1.538e+01 -1.391e-02 -4.291e+01]
 [ 3.436e-02  1.678e+03 -9.408e-02 -1.592e+01  2.630e-03  4.432e+01]
 [-8.766e-01  5.478e+00  2.431e+00 -2.370e-01 -7.218e-02  7.766e-02]
 [ 4.642e-03 -1.803e+03 -7.310e+00  2.016e+01  1.183e+00 -4.659e+01]
 [-3.097e-02 -1.678e+03  9.049e-02  1.557e+01 -3.293e-03 -4.338e+01]
 [-2.543e+00  1.733e+02  7.053e+00 -4.878e+00 -2.093e-01  3.300e+00]]


In [82]:
force = np.concatenate([[0, 0, 0], rotate(angles_rpy[-1], [0, 0, 1])])
jacobian_trans = jacobian.T
#torque = jacobian_inv.dot(force)
torque = jacobian_trans.dot(force)

print('Force vector: {}'.format(force))
print('Joint torques: {}'.format(torque))

Force vector: [ 0.     0.     0.    -0.324  0.939 -0.116]
Joint torques: [-9.388e-01  1.479e-03 -3.446e-01  9.388e-01  7.902e-05  1.000e+00]
